In [ ]:
!pip install accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub
!pip install langchain
!pip install PYPDF2
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install -U langchain-community
!pip install install sentence_transformers
!pip install llama_index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-langchain
!pip install llama-index

In [41]:
!pip install gensim


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
documents=SimpleDirectoryReader("/kaggle/input/demo-db").load_data()
documents

In [7]:
system_prompt="""
For each cited paper, determine if it's relevant to the source paper. 
Return True or False with a brief explanation.
Perform a semantic search between the source paper and other provided papers.
If two papers are semantically similar, explain why
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [12]:

import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)
  

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [14]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00 0:00:01


In [15]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

2024-08-13 21:07:55.676999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 21:07:55.677119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 21:07:55.812866: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_35/2250285859.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [17]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ea7d2a7a380>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ea7d2a7a380>, id_func=<function default_id_func at 0x7ea8a18b2e60>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7ea891995a50>, callback_manager=<llama_index

In [18]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)
     

In [19]:
index

In [20]:
query_engine=index.as_query_engine()

In [ ]:
!pip install PyPDF2

In [25]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    pdf_reader = PyPDF2.PdfReader(open(pdf_path, 'rb'))
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

pdf_path = '/kaggle/input/demo-db/t2speech.pdf'
text = extract_text_from_pdf(pdf_path)
print(text)

 
 A review -based  study  on different  Text-to-Speech  
technologies  
 
 
Md. Jalal Uddin  Chowdhury,  Ashab  Hussan  
Leading  Univ ersity,  jalalchy101,  ashabhtanim @gmail.com  
 
 
 
Abstract  - This  research  paper  presents  a comprehensive  
review -based  study  on various  Text -to-Speech  (TTS)  
technologies.  TTS  technology  is an important  aspect  of 
human -computer  interaction,  enabling  machines  to 
convert  written  text into audible  speech.  The paper  
exam ines the different  TTS  technologies  available,  
including  concatenative  TTS,  formant  synthesis  TTS,  and 
statistical  parametric  TTS.  The study  focuses  on 
comparing  the advantages  and limitations  of these  
technologies  in terms  of their  naturalness  of voice,  the 
level  of complexity  of the system,  and their  suitability  for 
different  applications.  In addition,  the paper  explores  the 
latest  advancements  in TTS  technology,  including  neural  
TTS  and hybrid  TTS.  Th

In [26]:
import re

def extract_reference(text, ref_heading="REFERENCES"):
  pattern = f"{ref_heading}(.*?)(?=(##?\s+.+\n|$))"
  match = re.search(pattern, text, flags=re.DOTALL)
  return match.group(1).strip() if match else None

In [27]:
ref_papers = extract_reference(text).split("\n")

In [28]:
ref_papers

['[1] Sadeque,  F. Y., Yasar,  S., & Islam,  M. M. (2013,  May).  Bangla  text to ',
 'speech  conversion:  A syllabic  unit selection  approach.  In 2013  International  ',
 'Conference  on Informatics,  Electronics  and Vision  (ICIEV)  (pp. 1-6). IEEE.  ',
 ' ',
 '[2] Firoj  Alam,  Promila  Kanti  Nath,  Mumit  Khan  (2007  ’Text  to speech  for ',
 'Bangla  language  using  festival’,  BRAC  University.  ',
 ' ',
 '[3] Firoj  Alam,  Promila  Kanti  Nath,  Mumit  Khan  (2011)  ‘Bangla  text to ',
 'speech  using  festival’,Conference  on human  language  technology  for ',
 'development,  pp.154 -161 ',
 ' ',
 '[4] Arafat,  M. Y., Fahrin,  S., Islam,  M. J., Siddiquee,  M. A., Khan,  A., ',
 'Kotwal,  M. R. A., & Huda,  M. N. (2014,  December).  Speech  synthesis  for ',
 'bangla  text to speech  conversion.  In The 8th International  Conference  on ',
 'Software,  Knowledge,  Information  Management  and Applications  (SKIMA  ',
 '2014)  (pp. 1-6). IEEE.  ',
 ' ',
 ' ',
 '[5] Ahmed

In [29]:

def papers_name(data):
  new_list = []
  current_list = []
  for item in data:
    if item == " ":
      if current_list:
        new_list.append(current_list)
      current_list = []
    else:
      current_list.append(item)
  if current_list:
    new_list.append(current_list)
  return new_list

In [30]:
title_papers = papers_name(ref_papers)

In [31]:
title_papers

[['[1] Sadeque,  F. Y., Yasar,  S., & Islam,  M. M. (2013,  May).  Bangla  text to ',
  'speech  conversion:  A syllabic  unit selection  approach.  In 2013  International  ',
  'Conference  on Informatics,  Electronics  and Vision  (ICIEV)  (pp. 1-6). IEEE.  '],
 ['[2] Firoj  Alam,  Promila  Kanti  Nath,  Mumit  Khan  (2007  ’Text  to speech  for ',
  'Bangla  language  using  festival’,  BRAC  University.  '],
 ['[3] Firoj  Alam,  Promila  Kanti  Nath,  Mumit  Khan  (2011)  ‘Bangla  text to ',
  'speech  using  festival’,Conference  on human  language  technology  for ',
  'development,  pp.154 -161 '],
 ['[4] Arafat,  M. Y., Fahrin,  S., Islam,  M. J., Siddiquee,  M. A., Khan,  A., ',
  'Kotwal,  M. R. A., & Huda,  M. N. (2014,  December).  Speech  synthesis  for ',
  'bangla  text to speech  conversion.  In The 8th International  Conference  on ',
  'Software,  Knowledge,  Information  Management  and Applications  (SKIMA  ',
  '2014)  (pp. 1-6). IEEE.  '],
 ['[5] Ahmed,  K. M., Ma

In [32]:
title_papers[0]

['[1] Sadeque,  F. Y., Yasar,  S., & Islam,  M. M. (2013,  May).  Bangla  text to ',
 'speech  conversion:  A syllabic  unit selection  approach.  In 2013  International  ',
 'Conference  on Informatics,  Electronics  and Vision  (ICIEV)  (pp. 1-6). IEEE.  ']

In [33]:
for i in range(3):
    paper = title_papers[i]
    print("="*30)
    print(paper)
    print("Citation Verfication \n")
    print("Yes / NO , explanation")
    citation =query_engine.query("Is this paper "+ str(paper) +" useful for the content "+ str(summary) + " answer yes or no with a small explanation")
    print(citation)
    print("="*30)
    print("\n")

['[1] Sadeque,  F. Y., Yasar,  S., & Islam,  M. M. (2013,  May).  Bangla  text to ', 'speech  conversion:  A syllabic  unit selection  approach.  In 2013  International  ', 'Conference  on Informatics,  Electronics  and Vision  (ICIEV)  (pp. 1-6). IEEE.  ']
Citation Verfication 

Yes / NO , explanation
 Yes, the paper ['[1] Sadeque,  F. Y., Yasar,  S., & Islam,  M. M. (2013,  May).  Bangla  text to speech conversion: A syllabic unit selection approach.  In 2013 International Conference on Informatics, Electronics and Vision (ICIEV) (pp. 1-6). IEEE. '] is relevant to the source paper as it also discusses the challenges and limitations of current methods and propose new approaches for improving the quality and naturalness of the synthesized speech. The paper also highlights the importance of considering the emotional and prosody in the synthesis process.


['[2] Firoj  Alam,  Promila  Kanti  Nath,  Mumit  Khan  (2007  ’Text  to speech  for ', 'Bangla  language  using  festival’,  BRAC  U

In [65]:
semantic_similarity =query_engine.query(''' Consider the "t2speech.pdf" as the source paper. Perform semantic similarity analysis on all other papers in the database. Provide the results in a table format where each row contains the name of the paper and the similarity score. The score should be a value ranging from 0 to 1, where 0 means no similarity is found. The papers to compare are:


- "design and implementation.pdf"
- "implementation-of-text-to-speech-conversion-IJERTV3IS030548.pdf"
- "t2s_synthesis.pdf"
- "t2v_visual_impaired.pdf"
''')

In [66]:
print(semantic_similarity)


Based on the provided context and without prior knowledge, I have performed a semantic similarity analysis on the source paper "t2speech.pdf" and the other papers in the database. The results are presented in a table format below:

| Paper Name | Similarity Score |
| --- | --- |
| design and implementation.pdf | 0.75 |
| implementation-of-text-to-speech-conversion-IJERTV3IS030548.pdf | 0.5 |
| t2s_synthesis.pdf | 0.25 |
| t2v_visual_impaired.pdf | 0.375 |

Explanation:

* The similarity score ranges from 0 to 1, where 0 means no similarity is found and 1 means maximum similarity.
* The similarity analysis is based on the content of the papers, including the keywords, phrases, and sentences.
* The similarity between the source paper "t2speech.pdf" and each other paper is calculated based on the frequency of shared keywords, phrases, and sentences.
* The similarity score is
